In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
import glob
import os
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import pickle

game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [29]:
def get_sats_games_from_prefixes(prefixes):
    game_list = []
    for prefix in prefixes:
        game_list += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    return sorted(set(game_list))

# can apply other filters like:
# sats_games = [s for s in sats_games if ('hide' not in s) and '1000' in s and 'risk_averse' not in s]
# sats_games = sats_games[:1] # debug

In [34]:
# feb4 4-type game; 11 seeds of CFR/CFR-no-tremble/PPO
sats_games = get_sats_games_from_prefixes(['feb4_4t'])
pgrid = ParameterGrid([{
    'game': [g for g in sats_games][:],
    'alg_and_config': [('cfr', 'cfr_port_10_ext'), ('ppo', 'ppo_jun8_23')]
}])
BASE_JOB_NAME = 'feb5_v2'
N_SEEDS = 11 # Number of seeds of each experiment to launch
time_limit_seconds = int(3600 * 1)
num_iters = 999_000_000
extra_name = ''
overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 1000 --time_limit_seconds {time_limit_seconds} --use_wandb true'

# ----------------------------------------------------

print(f'Games: {len(sats_games)}; Grid: {len(pgrid)}')
for params in pgrid:
    game_name = params['game']
    (alg_name, alg_config) = params['alg_and_config']
    dispatch.dispatch_experiments(alg_config, base_job_name=BASE_JOB_NAME, game_name=game_name, overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg=alg_name)

Games: 20; Grid: 40
Submitted batch job 1081224
Submitted batch job 1081225
Submitted batch job 1081226
Submitted batch job 1081227
Submitted batch job 1081228
Submitted batch job 1081229
Submitted batch job 1081230
Submitted batch job 1081231
Submitted batch job 1081232
Submitted batch job 1081233
Submitted batch job 1081234
Submitted batch job 1081235
Submitted batch job 1081236
Submitted batch job 1081237
Submitted batch job 1081238
Submitted batch job 1081239
Submitted batch job 1081240
Submitted batch job 1081241
Submitted batch job 1081242
Submitted batch job 1081243
Submitted batch job 1081244
Submitted batch job 1081245
Submitted batch job 1081246
Submitted batch job 1081247
Submitted batch job 1081248
Submitted batch job 1081249
Submitted batch job 1081250
Submitted batch job 1081251
Submitted batch job 1081252
Submitted batch job 1081253
Submitted batch job 1081254
Submitted batch job 1081255
Submitted batch job 1081256
Submitted batch job 1081257
Submitted batch job 1081258


KeyboardInterrupt: 

In [19]:
# other example jobs below

In [6]:
# ### Just GOF CFR
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 1)
# for config in pgrid:
#     CONFIG = 'cfr' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 1800 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 50 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jan18_cfr_v6_seeds_v3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr', start_seed=200)

In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1)
for config in pgrid:
    CONFIG = 'cfr_10' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb4_v1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 1079913


In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1 / 60)
for config in pgrid:
    CONFIG = 'ppo_jun8_23' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb3_test_short_v4', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='ppo')

Submitted batch job 1080222
Submitted batch job 1080223
Submitted batch job 1080224
Submitted batch job 1080225
Submitted batch job 1080226
Submitted batch job 1080227
Submitted batch job 1080228
Submitted batch job 1080229
Submitted batch job 1080230
Submitted batch job 1080231
Submitted batch job 1080232
Submitted batch job 1080233
Submitted batch job 1080234
Submitted batch job 1080235
Submitted batch job 1080236
Submitted batch job 1080237
Submitted batch job 1080238
Submitted batch job 1080239
Submitted batch job 1080240
Submitted batch job 1080241
Submitted batch job 1080242
Submitted batch job 1080243
Submitted batch job 1080244
Submitted batch job 1080245
Submitted batch job 1080246
Submitted batch job 1080247
Submitted batch job 1080248
Submitted batch job 1080249
Submitted batch job 1080250
Submitted batch job 1080251
Submitted batch job 1080252
Submitted batch job 1080253
Submitted batch job 1080254
Submitted batch job 1080255
Submitted batch job 1080256
Submitted batch job 

In [37]:
!squeue -u $USER --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)
           1080121                                                                                    jupyter  R .2-07:39:51     1 ada21


In [36]:
!scancel 1081263 1081264 1081265 1081266 1081267 1081268 1081269 1081270 1081271 1081272 1081273 1081274 1081275 1081276 1081277 1081278 1081279 1081280 1081281 1081282 1081283 1081284 1081285 1081286 1081287 1081288 1081289 1081290 1081291 1081292 1081293 1081294 1081295 1081296 1081297 1081298 1081299 1081300 1081301 1081302 1081303 1081304 1081305 1081306 1081307 1081308 1081309 1081310 1081311 1081312 1081313 1081314 1081315 1081316 1081317 1081318 1081319 1081320 1081321 1081322 1081323 1081324 1081325 1081326 1081327 1081328 1081329 1081330 1081331 1081332 1081333 1081334 1081335 1081336 1081337 1081338 1081339 1081340 1081341 1081342 1081343 1081344 1081345 1081346 1081347 1081348 1081349 1081350 1081351 1081352 1081353 1081354 1081355 1081356 1081357 1081358 1081359 1081360 1081361 1081362 1081363 1081364 1081365 1081366 1081367 1081368 1081369 1081370 1081371 1081372 1081373 1081374 1081375 1081376 1081377 1081378 1081379 1081380 1081381 1081382 1081383 1081384 1081385 1081260 1081261 1081262 1081252 1081253 1081254 1081255 1081256 1081257 1081258 1081259 1081243 1081244 1081245 1081246 1081247 1081248 1081249 1081250 1081251 1081234 1081235 1081236 1081237 1081238 1081239 1081240 1081241 1081242 1081226 1081227 1081228 1081229 1081230 1081231 1081232 1081233 1081224 1081225 1081223


In [35]:
!squeue -u $USER --format="%.18i %.65j %.2t %.10M %.6D %R" | grep -v "jupyter" | awk '{print $1}'

JOBID
1081263
1081264
1081265
1081266
1081267
1081268
1081269
1081270
1081271
1081272
1081273
1081274
1081275
1081276
1081277
1081278
1081279
1081280
1081281
1081282
1081283
1081284
1081285
1081286
1081287
1081288
1081289
1081290
1081291
1081292
1081293
1081294
1081295
1081296
1081297
1081298
1081299
1081300
1081301
1081302
1081303
1081304
1081305
1081306
1081307
1081308
1081309
1081310
1081311
1081312
1081313
1081314
1081315
1081316
1081317
1081318
1081319
1081320
1081321
1081322
1081323
1081324
1081325
1081326
1081327
1081328
1081329
1081330
1081331
1081332
1081333
1081334
1081335
1081336
1081337
1081338
1081339
1081340
1081341
1081342
1081343
1081344
1081345
1081346
1081347
1081348
1081349
1081350
1081351
1081352
1081353
1081354
1081355
1081356
1081357
1081358
1081359
1081360
1081361
1081362
1081363
1081364
1081365
1081366
1081367
1081368
1081369
1081370
1081371
1081372
1081373
1081374
1081375
1081376
1081377
1081378
1081379
1081380
1081381
1081382
1081383
1081384
1081385
1081260
10

In [9]:
!squeue -u $USER | wc -l

102


In [17]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u $USER --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [18]:
check_queue()

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
           1079910                                                                                    jupyter  R .     49:46      1 ada21
           1079915 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-1350985_who_cares_p  R .      1:27      1 ada01
           1079913 jan12_repro_4t_jan12_repro_4t_1_base_dev1000_rho1_t4-cfr_10external_plus_linear-100_who_ca  R .     36:32      1 ada01


In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [20]:
!scancel  1079920 

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)